# Génération de la Frontière Efficiente de Markowitz

Ce notebook illustre l'utilisation de la méthode de scalarisation pour approximer la frontière efficiente d'un portefeuille d'actions.


In [ ]:
import matplotlib.pyplot as plt
from scipy.constants import sigma
from scipy.optimize import minimize
import os
import pandas as pd
import cvxpy as cp
import numpy as np

from level1.functions import f_returns, f_mu, f_sigma

# Charger les données
df = pd.read_csv('../datasets/Information_Technology.csv', index_col=0, parse_dates=True)

for file in os.listdir('../datasets/'):
    if file.endswith('.csv') and 'Information_Technology' not in file:
        temp_df = pd.read_csv(os.path.join('../datasets/', file), index_col=0, parse_dates=True)
        df = df.join(temp_df, how='inner')

In [ ]:
df

In [ ]:
df.info()

In [ ]:
# Calcul des rendements logarithmiques
returns = f_returns(df)

# Calcul des paramètres pour l'optimisation
mu = f_mu(returns)  # Annualisation (252 jours boursiers)
Sigma = f_sigma(returns)  # Annualisation de la matrice de covariance
num_assets = len(mu)
mu = mu.values.astype(float)        # shape (196,)
Sigma = Sigma.values.astype(float)  # shape (196,196)

In [ ]:
print(type(mu), mu.shape)
print(type(Sigma), Sigma.shape)


In [ ]:
import cvxpy as cp
print(cp.installed_solvers())


# Calcul des Paramètres d'Optimisation

Les rendements logarithmiques sont calculés comme :

$  r_t = \ln\left(\frac{P_t}{P_{t-1}}\right) $

Le vecteur des rendements moyens annualisés :

$ \mu = \frac{1}{T} \sum_{t=1}^T r_t \times 252 $

La matrice de covariance annualisée :

$ \Sigma = \frac{1}{T} \sum_{t=1}^T (r_t - \bar{r})(r_t - \bar{r})^T \times 252 $


# Méthode de résolution par scalarisation pour générer la frontière efficiente

Fonction rendement : $ F_1(w) = - (w^T \mu) $

Fonction risque : $ F_2(w) = w^T \Sigma w $

Fonction de cout de transaction : $ C(w) = \sum_{i=1}^{N} c_{prop} \cdot |w_i - w_i^{prev}| $

Fonction objectif scalarisée : $ F(w) = \lambda \cdot (w^T \Sigma w) - (1 - \lambda) \cdot (w^T \mu) $

## Use brute force to find different cardinality portfolios

In [ ]:
from level2.cardinality_epsilon import optimize

K = 2                    # nombre d'actifs à sélectionner
epsilons = np.linspace(0.001, 1, 50)  # différents niveaux de risque

frontier_returns, frontier_volatilities, frontier_weights = optimize(mu, Sigma,K, epsilons)

In [ ]:
import pickle
with open(f'frontier_data_epsilon.pkl', 'wb') as f:
    pickle.dump((frontier_returns, frontier_volatilities, frontier_weights), f)

In [ ]:
plt.figure(figsize=(8,5))
plt.plot(frontier_volatilities, frontier_returns, marker='o')
plt.xlabel('Risque')
plt.ylabel('Rendement')
plt.title(f'Frontière de Pareto approx. K={K} actifs')
plt.grid(True)
plt.show()

# Génération de la Frontière Efficiente

In [ ]:
print("Le portefeuille avec le rendement le plus élevé :")
max_return_index = np.argmax(frontier_returns)
print(f"Rendement : {frontier_returns[max_return_index]:.4f}, Volatilité : {frontier_volatilities[max_return_index]:.4f}")
weights = frontier_weights[max_return_index]
weights[weights < 1e-4] = 0  # Nettoyer les poids très faibles pour l'affichage
print(f"Actifs sélectionnés :")
for i, weight in enumerate(weights):
    if weight > 0:
        print(f"  {df.columns[i]} : {weight:.4f}")
#print(f"Poids : {weights}")

print("\nLe portefeuille avec le risque le plus faible :")
min_risk_index = np.argmin(frontier_volatilities)
print(f"Rendement : {frontier_returns[min_risk_index]:.4f}, Volatilité : {frontier_volatilities[min_risk_index]:.4f}")
weights = frontier_weights[min_risk_index]
weights[weights < 1e-4] = 0  # Nettoyer les poids très faibles pour l'affichage
print(f"Actifs sélectionnés :")
for i, weight in enumerate(weights):
    if weight > 0:
        print(f"  {df.columns[i]} : {weight:.4f}")
#print(f"Poids : {weights}")

est ce que on peut pas relaxer une conteainte de cardinalité par une pénalisation L1 ? Si oui sous quelle condition ?
Oui, il est possible de relaxer une contrainte de cardinalité en utilisant une pénalisation L1, mais cela dépend de certaines conditions. La contrainte de cardinalité impose une limite sur le nombre d'actifs non nuls dans un portefeuille, ce qui est une contrainte non convexe et difficile à gérer directement dans les problèmes d'optimisation. En revanche, la pénalisation L1 (ou régularisation Lasso) encourage la sparsité dans les solutions en ajoutant une pénalité proportionnelle à la somme des valeurs absolues des coefficients (poids des actifs dans le portefeuille).